In [ ]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout,Conv1D,GlobalMaxPooling1D # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/SetModels/IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [ ]:
english_stops = stopwords.words('english')
print(english_stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words and spliting
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
5636     [i, watched, almost, gundam, mech, anime, show...
25192    [i, love, bad, movies, not, often, entertainin...
4748     [given, title, outlandish, box, art, i, ready,...
7084     [this, one, seriously, disturbed, movie, even,...
1831     [although, like, cricket, seen, movie, years, ...
                               ...                        
26611    [we, bought, dvd, walking, dinosaurs, nearly, ...
37513    [i, watched, film, beginning, end, i, promised...
8405     [when, i, i, saw, documentary, a, funny, thing...
18798    [i, going, tell, anyone, happens, end, fit, mo...
39779    [yes, thing, film, memorable, starred, youngis...
Name: review, Length: 40000, dtype: object 

17332    [hello, i, wondering, anyone, copy, movie, bro...
43534    [the, regard, flight, written, performed, bill...
27265    [you, realize, watching, exact, same, show, ei...
27260    [william, hurt, scuba, diving, scientist, us, ...
26133    [surprisingly, enough, movie, redeeming, quali...
 

In [ ]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [ ]:
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()
print(max_length)
x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

130
Encoded X Train
 [[   1  195  119 ...    0    0    0]
 [   1   40   19 ...  100 1247  587]
 [ 261  327 7695 ...    0    0    0]
 ...
 [ 171    1    1 ...   62 1677  735]
 [   1   80  284 ...    0    0    0]
 [ 320   66    4 ... 2525    6 3191]] 

Encoded X Test
 [[ 4798     1  1529 ...     0     0     0]
 [    2  2842  2488 ...     0     0     0]
 [   97   863    65 ... 14079   689  3371]
 ...
 [ 2342  2957   305 ...     0     0     0]
 [    8   270   530 ...     0     0     0]
 [    1   337     3 ...     0     0     0]] 

Maximum review length:  130


In [ ]:
EMBED_DIM = 32

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(Conv1D(filters = 64, kernel_size = 3, strides= 1, padding='same', activation= 'relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(units = 256, activation= 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           2947872   
                                                                 
 conv1d (Conv1D)             (None, 130, 64)           6208      
                                                                 
 global_max_pooling1d (Glob  (None, 64)                0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                        

In [ ]:
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size = 128, epochs = 5)

Epoch 1/5
313/313 [==============================] - 29s 87ms/step - loss: 0.4479 - accuracy: 0.7742 - val_loss: 0.3106 - val_accuracy: 0.8671
Epoch 2/5
313/313 [==============================] - 25s 79ms/step - loss: 0.2056 - accuracy: 0.9204 - val_loss: 0.2932 - val_accuracy: 0.8763
Epoch 3/5
313/313 [==============================] - 25s 81ms/step - loss: 0.0664 - accuracy: 0.9810 - val_loss: 0.3686 - val_accuracy: 0.8679
Epoch 4/5
313/313 [==============================] - 25s 80ms/step - loss: 0.0138 - accuracy: 0.9976 - val_loss: 0.4087 - val_accuracy: 0.8754
Epoch 5/5
313/313 [==============================] - 25s 81ms/step - loss: 0.0024 - accuracy: 0.9999 - val_loss: 0.4464 - val_accuracy: 0.8750


In [ ]:
y_pred =  (model.predict(x_test) > 0.7).astype("int32")

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

313/313 [==============================] - 1s 4ms/step
Correct Prediction: 8750
Wrong Prediction: 1250
Accuracy: 87.5


In [ ]:
model.save('CNN_IMDB_T1.keras')



In [ ]:
loaded_model = load_model('CNN_IMDB_T1.keras')

In [ ]:
review = str(input('Movie Review: '))

Movie Review:  It was good


In [ ]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  It was good
Filtered:  ['it good']


In [ ]:
   # no need lower, because already lowered the data in load_data()
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length , padding='post', truncating='post')
print(tokenize_words)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 [==============================] - 0s 69ms/step
[[0.06568262]]


In [ ]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

negative
